In [1]:
import json

file_name='配电施工'
def get_json(path:str)->dict:
    """
    获取json文件
    """
    with open(path, 'r', encoding='utf-8') as file:
        data = json.load(file)
    return data['result']

tk=get_json(f"./data/工作票/{file_name}.json")

In [3]:
def get_safety(safe_list:list[dict],code:str):
    '''
    第一工作票，根据安全措施类型代码获取安全措施列表
    '''
    filter_list=[item for item in safe_list if item['conOfSecurityMeasuresTypeCode']==code]
    return sorted(filter_list,key=lambda x:x['lineNo'])

def get_person_change(person_list:list[dict]):
    '''
    配电施工，获取新增人员
    '''
    filter_list=[item for item in person_list if item['addStaffSign']!=""]
    return sorted(filter_list,key=lambda x:x['addStaffSignTime'])

In [4]:
def find_paths(data, target):
    """
    查询json路径函数
    """
    paths = []
    def recursive_search(obj, path="tk"):
        if isinstance(obj, dict):
            for k, v in obj.items():
                new_path = f'{path}["{k}"]'
                if str(v).startswith(target):
                    print(v)
                    paths.append(new_path)
                else:
                    recursive_search(v, new_path)
        elif isinstance(obj, list):
            for idx, item in enumerate(obj):
                new_path = f"{path}[{idx}]"
                recursive_search(item, new_path)

    recursive_search(data)
    return paths

# paths = find_paths(tk,target='10kV里金555线93')
# for p in paths:
#     print(p)

In [6]:

#模板
template=f'''
单位：{tk["ConstructionWorkTicket"][0]["workUnit"]}
编号：{tk["ConstructionWorkTicket"][0]["workTicketNo"]}

1.作业负责人：{tk["ConstructionWorkTicket"][0]["workingControllerName"]} 施工班组：{tk["ConstructionWorkTicket"][0]["workTeam"]}

2.作业班成员：
{tk["ConstructionWorkTicket"][0]["workingPerson"]}
共{len(tk["ConstructionWorkTicket"][0]["workingPerson"])}人
人员分工：
{tk["ConstructionWorkTicket"][0]["workCondition"]}

3.作业任务
|作业地点 |作业内容|
|--------|----|
{'\n'.join([f'|{x['workPlace']}|{x['workContent']}|' for x in tk["SGTaskList"]])}

4.计划工作时间：自{tk["ConstructionWorkTicket"][0]["plannedStartTime"]}至 {tk["ConstructionWorkTicket"][0]["plannedEndTime"]}

5.作业必备检查条件：
|检查内容|是否检查|
|-------|-----|
|作业人员|{tk["ConstructionWorkTicket"][0]["isAffirm"].split(',')[0]}|
|施工机械|-----|
|-------|-----|
|-------|-----|
|-------|-----|
|-------|-----|

6. 作业过程预控措施及落实：
{tk["SGSafetyMeasures"][0]["contentOfSecurityMeasures"]}
作业票签发人：{tk["ConstructionWorkTicket"][0]["firstSignature"]}           时间：{tk["ConstructionWorkTicket"][0]["firstSignatureTime"]}
工作负责人：{tk["ConstructionWorkTicket"][0]["pickupTicketName"]}      时间：{tk["ConstructionWorkTicket"][0]["pickupticketTime"]}    

7.现场交底。作业班成员确认作业负责人布置的作业任务，人员分工，安全措施和注意事项并签名：
{tk["ConstructionWorkTicket"][0]["workingpersonSign"]}
开工时间：{tk["ConstructionWorkTicket"][0]["ActualStartTime"]}               作业负责人：{tk["ConstructionWorkTicket"][0]["workingControllerName"]}

8.现场变化情况及补充安全措施：
{tk["SGSafetyMeasures"][1]["contentOfSecurityMeasures"]}

9.人员变更
9.1作业人员变更情况：
|新增人员姓名|时间|
|-------|-----|
{'\n'.join([f'|{x['addStaffSign']}|{x['addStaffSignTime']}|' for x in get_person_change(tk["SGPersonnelChangeDetailRecords"])])}

|离开人员姓名|时间|
|-------|-----|

9.2 作业负责人变更情况

11.作业终结
终结时间：{tk["ConstructionWorkTicket"][0]["ActualEndTime"]}       作业负责人：{tk["ConstructionWorkTicket"][0]["workingControllerName"]}

12 专责监护人
|负责监护（人员、地点及具体工作）|专责监护人|
|-------|-----|

13 备注：
{tk["ConstructionWorkTicket"][0]["remarks"]}
'''
rule='''
1. 现场勘察日期和 工作票执行日期（位于文档左上角）的间隔小于或等于15天。
2. 勘察负责人是 【工作负责人】或 工作票签发人。
3. 【工作班成员】。如果人数在5人内（包括），需填写所有人员姓名。如果在5人以上，只写5人姓名，并加“等”。
4. 【停电线路】。需要有 电压等级（数字+电压单位，例如，10kV，区分大小写）、线路双重名称（例如，季宅 B918线）。
5. 【工作任务】。需要有 电压等级（10kV，区分大小写）、线路及设备双重名称（季宅 B918线）、起止杆号（14杆）
6. 【时间逻辑】。*计划工作时间、许可时间、工作负责人下令开始工作时间、工作班装设、拆除接地线的时间、工作终结时间*需要满足时间先后关系。
7. 简图的规范性。略过。
8. 安全措施。略过。
9. 【交底人员签名】。签名人员 和人数 需要与【工作班成员】一致。
10. 【工作许可】。许可方式必须为‘当面许可’。许可内容 需详细程度到具体杆号（例如，季宅B918线外垄支线1号）。工作许可人、工作负责人姓名 需要一致。
11. 【接地线】。工作班现场装拆的接地线（个人保安线），数量是否匹配。
'''


In [7]:

def save_txt(data, path:str)->None:
    """
    保存txt文件
    """
    with open(path, 'w', encoding='utf-8') as file:
        file.write(data)

save_txt(template, f'./data/工作票/{file_name}/instance.txt')
save_txt(rule, f'./data/工作票/{file_name}/rule.txt')